In [27]:
import pyperclip

def copy_generated_autoencoder(layer_dimensions: list[int], flatten: bool = False) -> None:
    incr = sorted(layer_dimensions)
    decr = sorted(layer_dimensions, reverse=True)

    flattening_row = """
            nn.Flatten(start_dim=1),""" if flatten else ""
    
    deflattening_row = """
            nn.Unflatten(dim=1, unflattened_size=(field_shape[0], field_shape[1]))""" if flatten else ""

    result = f"""class AutoEncoder(nn.Module):

    def __init__(self, field_shape: int):
        super(AutoEncoder, self).__init__()
        self.field_shape = field_shape
        
        self.encoder = nn.Sequential({flattening_row}
            nn.Linear(field_shape[0] * field_shape[1], {decr[0]}),
            nn.ReLU(),"""
    for first, second in zip(decr, decr[1:]):
        result += f"""
            nn.Linear({first}, {second}),
            nn.ReLU(),"""
    result += """
        )
        self.decoder = nn.Sequential("""
    for first, second in zip(incr, incr[1:]):
        result += f"""
            nn.Linear({first}, {second}),
            nn.ReLU(),"""
    result += f"""
            nn.Linear({decr[0]}, field_shape[0] * field_shape[1]),
            nn.ReLU(),{deflattening_row}
        )
    
    def forward(self, x):
        encode = self.encoder(x)
        decode = self.decoder(encode)
        return decode
    """

    # copy to clipboard
    pyperclip.copy(result)

    print(f"Generated autoencoder (with layer dimensions: {incr}) copied to the clipboard")

In [28]:
import torch.nn as nn

def generate_autoencoder_model(layer_dimensions: list[int], field_shape, flatten: bool = False) -> nn.Module:
    incr = sorted(layer_dimensions)
    decr = sorted(layer_dimensions, reverse=True)

    encoder_layers = [nn.Flatten(start_dim=1)] if flatten else []
    encoder_layers += [nn.Linear(field_shape[0] * field_shape[1], decr[0]),
                      nn.ReLU()]
    for first, second in zip(decr, decr[1:]):
        encoder_layers += [nn.Linear(first, second),
                           nn.ReLU()]
    
    decoder_layers = []
    for first, second in zip(incr, incr[1:]):
        decoder_layers += [nn.Linear(first, second),
                           nn.ReLU()]
    decoder_layers += [nn.Linear(decr[0], field_shape[0] * field_shape[1]),
                       nn.ReLU()]
    if flatten:
        decoder_layers += [nn.Unflatten(dim=1, unflattened_size=(field_shape[0], field_shape[1]))]

    class AutoEncoder(nn.Module):

        def __init__(self):
            super(AutoEncoder, self).__init__()
            self.encoder = nn.Sequential(*encoder_layers)
            self.decoder = nn.Sequential(*decoder_layers)
        
        def forward(self, x):
            encode = self.encoder(x)
            decode = self.decoder(encode)
            return decode
    
    return AutoEncoder

In [29]:
# TODO insert in the following list the dimensions of the layers (one int for wanted layer) in any order
layer_dimensions = [2**i for i in range(3, 10)]
#layer_dimensions = [50000, 10000, 2500, 500, 10]

copy_generated_autoencoder(layer_dimensions, flatten=True)

Generated autoencoder (with layer dimensions: [8, 16, 32, 64, 128, 256, 512]) copied to the clipboard


In [30]:
AutoEncoder = generate_autoencoder_model(layer_dimensions, field_shape=[500, 200], flatten=True)
ananas = AutoEncoder()
ananas

AutoEncoder(
  (encoder): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=100000, out_features=512, bias=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): ReLU()
    (5): Linear(in_features=256, out_features=128, bias=True)
    (6): ReLU()
    (7): Linear(in_features=128, out_features=64, bias=True)
    (8): ReLU()
    (9): Linear(in_features=64, out_features=32, bias=True)
    (10): ReLU()
    (11): Linear(in_features=32, out_features=16, bias=True)
    (12): ReLU()
    (13): Linear(in_features=16, out_features=8, bias=True)
    (14): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=8, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=128, bias=True)
    (7): ReLU()
    (8): Linear(in_features=128, out_feat